# WhatsApp Message Importance Recommendation System
This project modifies a BERT-style encoder-only Transformer to classify WhatsApp-like messages as **Important** or **Not Important** using a synthetic dataset.

## Step 1: Install & Import Libraries

In [ ]:

!pip install transformers torch scikit-learn pandas
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
    

## Step 2: Load Synthetic WhatsApp Dataset

In [ ]:

# Load dataset
df = pd.read_csv("whatsapp_messages.csv")
print(df.head())

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["message"].values, df["label"].values, test_size=0.2, random_state=42
)
    

## Step 3: Tokenization

In [ ]:

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, return_tensors="pt")

# Convert labels to integers
label_map = {"not_important": 0, "important": 1}
train_labels = torch.tensor([label_map[label] for label in train_labels])
test_labels = torch.tensor([label_map[label] for label in test_labels])
    

## Step 4: Define Transformer-based Classifier

In [ ]:

class ImportanceClassifier(nn.Module):
    def __init__(self, hidden_size=768, num_labels=2):
        super(ImportanceClassifier, self).__init__()
        self.encoder = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # CLS token
        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)
        return logits

model = ImportanceClassifier()
    

## Step 5: Training Setup

In [ ]:

train_dataset = TensorDataset(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels)
test_dataset = TensorDataset(test_encodings["input_ids"], test_encodings["attention_mask"], test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
    

## Step 6: Training Loop

In [ ]:

epochs = 2
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")
    

## Step 7: Evaluation

In [ ]:

model.eval()
all_preds, all_labels = [], []

for batch in test_dataset:
    input_ids, attention_mask, labels = [b.unsqueeze(0).to(device) for b in batch]
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    all_preds.extend(preds)
    all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=["not_important", "important"]))
    